In [1]:
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data = pd.read_excel('top_clgs_new.xlsx')
data.head()

,College Name,College City,State,Approvals,Placement,Cut_Off,List_of_Courses
0,IIM Bangalore - Indian Institute of Management,Bangalore,Karnataka,"['AICTE', 'UGC']","[{'Batch': 'Number of Companies Participated',...","[{'College': 'IIM Ahmedabad', 'Minimum Expecte...",NaN
1,IIT Madras - Indian Institute of Technology - ...,Chennai,Tamil Nadu,['AICTE'],"[{'Batch': 'Number of Companies Participated',...","[{('Courses', 'Courses'): 'B.Tech Computer Sci...","[{0: 'Program', 1: 'Eligibility', 2: 'IIT Madr..."
2,Universal Business School - [UBS],Karjat,Maharashtra,"['AICTE', 'NBA', 'AIU']",[{'Particulars': 'Highest International packag...,"[{'Name of the Exam': 'CAT', 'Cutoff': '70%ile...",NaN
3,IIT Delhi - Indian Institute of Technology [IITD],New Delhi,Delhi NCR,[],"[{'Particulars': 'PPOs Offered', 'Placement St...","[{('B.Tech Specialisation', 'B.Tech Specialisa...","[{('IIT Delhi Courses & Fees 2023', 'Program')..."
4,IIT Bombay - Indian Institute of Technology - ...,Mumbai,Maharashtra,"['AICTE', 'UGC']",[{'Particulars': 'No. of Companies offered Job...,"[{('Courses', 'Courses'): 'B.Tech Computer Sci...",NaN


In [3]:
data.columns

Index(['College Name', 'College City', 'State', 'Approvals', 'Placement',
       'Cut_Off', 'List_of_Courses'],
      dtype='object')

In [4]:
data.shape

(10010, 7)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10010 entries, 0 to 10009
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   College Name     10010 non-null  object
 1   College City     10010 non-null  object
 2   State            10010 non-null  object
 3   Approvals        10010 non-null  object
 4   Placement        1086 non-null   object
 5   Cut_Off          4136 non-null   object
 6   List_of_Courses  749 non-null    object
dtypes: object(7)
memory usage: 547.5+ KB


In [6]:
data.isnull().sum()

College Name          0
College City          0
State                 0
Approvals             0
Placement          8924
Cut_Off            5874
List_of_Courses    9261
dtype: int64

In [7]:
data.fillna(' ')

,College Name,College City,State,Approvals,Placement,Cut_Off,List_of_Courses
0,IIM Bangalore - Indian Institute of Management,Bangalore,Karnataka,"['AICTE', 'UGC']","[{'Batch': 'Number of Companies Participated',...","[{'College': 'IIM Ahmedabad', 'Minimum Expecte...",
1,IIT Madras - Indian Institute of Technology - ...,Chennai,Tamil Nadu,['AICTE'],"[{'Batch': 'Number of Companies Participated',...","[{('Courses', 'Courses'): 'B.Tech Computer Sci...","[{0: 'Program', 1: 'Eligibility', 2: 'IIT Madr..."
2,Universal Business School - [UBS],Karjat,Maharashtra,"['AICTE', 'NBA', 'AIU']",[{'Particulars': 'Highest International packag...,"[{'Name of the Exam': 'CAT', 'Cutoff': '70%ile...",
3,IIT Delhi - Indian Institute of Technology [IITD],New Delhi,Delhi NCR,[],"[{'Particulars': 'PPOs Offered', 'Placement St...","[{('B.Tech Specialisation', 'B.Tech Specialisa...","[{('IIT Delhi Courses & Fees 2023', 'Program')..."
4,IIT Bombay - Indian Institute of Technology - ...,Mumbai,Maharashtra,"['AICTE', 'UGC']",[{'Particulars': 'No. of Companies offered Job...,"[{('Courses', 'Courses'): 'B.Tech Computer Sci...",
...,...,...,...,...,...,...,...
10005,Katuri Medical College and Hospital,Guntur,Andhra Pradesh,['MCI'],,"[{'Courses': nan, 'General': nan}, {'Courses':...","[{'Course': 'B.Sc. Nursing', 'Details': 'Durat..."
10006,Kempegowda Institute of Medical Sciences - [KIMS],Bangalore,Karnataka,['MCI'],,,
10007,Kempegowda Institute of Physiotherapy - [KIPT],Bangalore,Karnataka,['IAP'],,,
10008,Khaja Bandanawaz Institute of Medical Sciences...,Gulbarga,Karnataka,['MCI'],,,


In [8]:
selecting_features = ['College Name', 'College City', 'State', 'Approvals', 'Placement',
       'Cut_Off', 'List_of_Courses'];
for feature in selecting_features:
    data[feature] = data[feature].fillna('')
    

In [9]:
combine_feat = data['College Name']+' '+data['College City']+' '+data['State']+' '+data['Approvals']+' '+data['Placement'] +data['Cut_Off'] + data['List_of_Courses']

In [10]:
print(combine_feat)

0        IIM Bangalore - Indian Institute of Management...
1        IIT Madras - Indian Institute of Technology - ...
2        Universal Business School - [UBS] Karjat Mahar...
3        IIT Delhi - Indian Institute of Technology [II...
4        IIT Bombay - Indian Institute of Technology - ...
                               ...                        
10005    Katuri Medical College and Hospital Guntur And...
10006    Kempegowda Institute of Medical Sciences - [KI...
10007    Kempegowda Institute of Physiotherapy - [KIPT]...
10008    Khaja Bandanawaz Institute of Medical Sciences...
10009    Government Kilpauk Medical College - [GKMC] Ch...
Length: 10010, dtype: object


In [11]:
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combine_feat)

In [12]:
print(feature_vectors)

  (0, 50659)	0.01546680735099763
  (0, 57519)	0.0179395608192362
  (0, 60913)	0.026861304953291813
  (0, 65951)	0.027736929718885608
  (0, 55473)	0.013330162678881178
  (0, 56695)	0.025627181160947452
  (0, 50931)	0.03170034338722109
  (0, 58410)	0.016858001816220613
  (0, 60343)	0.01574970964409265
  (0, 59803)	0.021049185018320583
  (0, 54238)	0.02304824051279835
  (0, 70)	0.03286768420612785
  (0, 60833)	0.21404044080416182
  (0, 51461)	0.12344798738311738
  (0, 55240)	0.2800479120744623
  (0, 54324)	0.27256534440346
  (0, 56474)	0.3926541009681719
  (0, 61193)	0.1206217857853921
  (0, 52139)	0.015769538533649395
  (0, 54892)	0.04864193899638121
  (0, 61120)	0.017820183801862205
  (0, 52413)	0.014202813605662872
  (0, 62836)	0.020938491954860194
  (0, 54927)	0.011456983885136442
  (0, 53179)	0.02475155639535365
  :	:
  (10009, 60116)	0.021492058534045202
  (10009, 67645)	0.021318280031249227
  (10009, 55511)	0.02069834757460608
  (10009, 60955)	0.09123046666444545
  (10009, 54922)	0

In [13]:
similarity = cosine_similarity(feature_vectors)
print(similarity)

[[1.00000000e+00 2.80711010e-01 3.88749581e-01 ... 1.45599083e-02
  4.90471894e-03 3.02632224e-04]
 [2.80711010e-01 1.00000000e+00 2.10833561e-01 ... 6.13572920e-03
  1.39033053e-02 8.41712967e-02]
 [3.88749581e-01 2.10833561e-01 1.00000000e+00 ... 3.52804811e-03
  3.15059659e-03 5.35201953e-05]
 ...
 [1.45599083e-02 6.13572920e-03 3.52804811e-03 ... 1.00000000e+00
  5.50536704e-02 0.00000000e+00]
 [4.90471894e-03 1.39033053e-02 3.15059659e-03 ... 5.50536704e-02
  1.00000000e+00 7.81731876e-03]
 [3.02632224e-04 8.41712967e-02 5.35201953e-05 ... 0.00000000e+00
  7.81731876e-03 1.00000000e+00]]


In [14]:
print(similarity.shape)

(10010, 10010)


In [15]:
user_choice = input('Enter the state where you want your college')

In [16]:
state_name = data['State'].tolist()
college_name = data['College Name'].tolist()
print(state_name)
print(college_name)

['Karnataka', 'Tamil Nadu', 'Maharashtra', 'Delhi NCR', 'Maharashtra', 'West Bengal', 'Uttar Pradesh', 'Uttarakhand', 'Rajasthan', 'Assam', 'Delhi NCR', 'Gujarat', 'Chandigarh', 'Karnataka', 'Rajasthan', 'Delhi NCR', 'Telangana', 'Delhi NCR', 'West Bengal', 'Delhi NCR', 'Tamil Nadu', 'Delhi NCR', 'Uttar Pradesh', 'Tamil Nadu', 'Uttar Pradesh', 'Delhi NCR', 'Delhi NCR', 'Uttar Pradesh', 'Tamil Nadu', 'Karnataka', 'Puducherry', 'Tamil Nadu', 'Gujarat', 'Chandigarh', 'Karnataka', 'Delhi NCR', 'Telangana', 'Delhi NCR', 'West Bengal', 'Delhi NCR', 'Tamil Nadu', 'Delhi NCR', 'Uttar Pradesh', 'Tamil Nadu', 'Uttar Pradesh', 'Delhi NCR', 'Delhi NCR', 'Uttar Pradesh', 'Tamil Nadu', 'Karnataka', 'Puducherry', 'Tamil Nadu', 'Andhra Pradesh', 'Delhi NCR', 'Uttar Pradesh', 'Chandigarh', 'Delhi NCR', 'Telangana', 'Delhi NCR', 'Uttar Pradesh', 'Maharashtra', 'Madhya Pradesh', 'Tamil Nadu', 'Jharkhand', 'Madhya Pradesh', 'Tamil Nadu', 'Delhi NCR', 'Tamil Nadu', 'Delhi NCR', 'Delhi NCR', 'Delhi NCR', 'T

In [17]:
find_close_match = difflib.get_close_matches(user_choice, state_name)
print(find_close_match)

['Kerala', 'Kerala', 'Kerala']


In [18]:
close_match = find_close_match[0]
print(close_match)

Kerala


In [19]:
name_of_college = data[data.State == close_match]['College Name'].values

In [20]:
print(name_of_college)

['Indian Institute of Management - [IIMK]' 'Trivandrum University College'
 'Mar Ivanios College' 'Sacred Heart College - [SH]'
 'Rajagiri College of Social Sciences - [RCSS]'
 'Manorama School of Communication - [MASCOM]'
 'National Institute of Technology - [NITC]'
 'Indian Institute of Space Science and Technology - [IIST]'
 'Amrita School of Dentistry' 'Mahatma Gandhi University - [MGU]'
 "St Thomas' College" "St. Albert's College" 'Calicut University - [CU]'
 'Amrita School of Medicine' 'St. Berchmans College - [SBC]'
 "St Teresa's College (Autonomous)"
 'St. Gregorios Dental College - [SGDC]'
 'National University of Advanced Legal Studies - [NUALS]'
 'Mahatma Gandhi College' 'Farook College'
 'Cochin University of Science and Technology, School of Legal Studies - [SLS]'
 'Government Law College' 'Newman College'
 'T.K.M. College of Arts and Science - [TKMCAS]' 'College of Engineering'
 "St Joseph's College Devagiri" 'Fatima Mata National College'
 'Oriental School of Hotel Manag

In [21]:
len(name_of_college)

577

In [22]:
print('Colleges recommended for you : \n')

i = 0

for movie in name_of_college:
  index = movie[0]
  name_of_college = data[data.State == close_match]['College Name'].values[i]
  if (i<30):
    print(i+1, '.',name_of_college)
    i+=1

Colleges recommended for you : 

1 . Indian Institute of Management - [IIMK]
2 . Trivandrum University College
3 . Mar Ivanios College
4 . Sacred Heart College - [SH]
5 . Rajagiri College of Social Sciences - [RCSS]
6 . Manorama School of Communication - [MASCOM]
7 . National Institute of Technology - [NITC]
8 . Indian Institute of Space Science and Technology - [IIST]
9 . Amrita School of Dentistry
10 . Mahatma Gandhi University - [MGU]
11 . St Thomas' College
12 . St. Albert's College
13 . Calicut University - [CU]
14 . Amrita School of Medicine
15 . St. Berchmans College - [SBC]
16 . St Teresa's College (Autonomous)
17 . St. Gregorios Dental College - [SGDC]
18 . National University of Advanced Legal Studies - [NUALS]
19 . Mahatma Gandhi College
20 . Farook College
21 . Cochin University of Science and Technology, School of Legal Studies - [SLS]
22 . Government Law College
23 . Newman College
24 . T.K.M. College of Arts and Science - [TKMCAS]
25 . College of Engineering
26 . St Jose

In [26]:
# the recommendation system
user_choice = input('Enter the state where you want your college')
find_close_match = difflib.get_close_matches(user_choice, state_name)
close_match = find_close_match[0]
name_of_college = data[data.State == close_match]['College Name'].values

print('User\'s choice of preference for State is : ',user_choice)

print('Colleges recommended for user : \n')

i = 0
for movie in name_of_college:
  index = movie[0]
  name_of_college = data[data.State == close_match]['College Name'].values[i]
  if (i<15):
    print(i+1, '.',name_of_college)
    i+=1

User's choice of preference for State is :  Chhattisgarh
Colleges recommended for user : 

1 . Indian Institute of Management - [IIM]
2 . Hidayatullah National Law University - [HNLU]
3 . National Institute of Technology - [NIT]
4 . Dr. C.V. Raman University - [CVRU]
5 . Government Jamuna Prasad Verma Post Graduate Arts and Commerce College
6 . Indira Gandhi Krishi Vishwavidyalaya - [IGKV]
7 . Guru Ghasidas Vishwavidyalaya - [GGU]
8 . Dignity College of Architecture - [DCA]
9 . ITM University
10 . Shaheed Gundadhur College of Agriculture and Research - [SGCARS]
11 . Mahant Laxminarayan Das College
12 . Pt. Ravishankar Shukla University - [PRSU]
13 . RMD College of Agriculture and Research Station
14 . Kushabhau Thakre Patrakarita Avam Jansanchar Vishwavidyalaya
15 . Chhattisgarh Kamdhenu Vishwavidyalaya - [CGKV]
